In [9]:
from typeconf import SelectConfig, BaseConfig
from pydantic import ValidationError
import torch
from torch import nn
from typing import Tuple, Union

Access predefined

In [8]:
from typeconf.libs.torch.optim import OptimizerConfig

In [16]:
class NonLinearityConfig(SelectConfig):
    pass

@NonLinearityConfig.register('sigmoid')
class Sigmoid(NonLinearityConfig):
    def build(self):
        return torch.sigmoid

@NonLinearityConfig.register('tanh')
class Tanh(NonLinearityConfig):
    def build(self):
        return torch.tanh

In [63]:
class ModelConfig(SelectConfig):
    pass

class ConvModel(nn.Module):
    def __init__(self, activation_fn, kernel_size):
        super().__init__()
        self.activation_fn = activation_fn
        self.conv1 = nn.Conv2d(3, 3, kernel_size=kernel_size)
    def forward(self, x):
        x = self.conv1(x)
        return self.activation_fn(x)

@ModelConfig.register('ConvNetwork')
class ConvModelConfig(ModelConfig):
    activation_fn : NonLinearityConfig
    kernel_size : Union[Tuple[int, int], int] = (3, 3)
    def build(self):
        activation_fn = self.activation_fn.build()
        return ConvModel(activation_fn, self.kernel_size)

class LinearModel(nn.Module):
    def __init__(self, activation_fn, in_features, out_features):
        super().__init__()
        self.activation_fn = activation_fn
        self.fc1 = nn.Linear(in_features, out_features)
    def forward(self, x):
        x = self.fc1(x)
        return self.activation_fn(x)

@ModelConfig.register('LinearNetwork')
class LinearModelConfig(ModelConfig):
    activation_fn : NonLinearityConfig
    in_features : int = 10
    out_features : int = 10
    def build(self):
        activation_fn = self.activation_fn.build()
        return LinearModel(activation_fn, self.in_features, self.out_features)

In [67]:
class ExperimentConfig(BaseConfig):
    model : ModelConfig
    optimizer : OptimizerConfig

In [85]:
cfg1 = {
    'model': {
        'name': 'LinearNetwork',
        'activation_fn': {
            'name': 'sigmoid'
        }
    },
    'optimizer': {
        'name': 'Adagrad'
    }
}

cfg2 = {
    'model': {
        'name': 'ConvNetwork',
        'activation_fn': {
            'name': 'tanh'
        }
    },
    'optimizer': {
        'name': 'Adagrad'
    }
}

# avoid runtime error because option does not exist
cfg_wrong_model_name = {
    'model': {
        'name': 'ConNetwork',
        'activation_fn': {
            'name': 'tanh'
        }
    },
    'optimizer': {
        'name': 'Adadelta'
    }
}

# avoid runtime error because of invalid data type
cfg_wrong_kernel_size = {
    'model': {
        'name': 'ConvNetwork',
        'kernel_size': (3, 3, 3),
        'activation_fn': {
            'name': 'tanh'
        }
    },
    'optimizer': {
        'name': 'Adagrad'
    }
}

# Avoid default value being used because wrong naming
cfg_wrong_kernel_size_name = {
    'model': {
        'name': 'ConvNetwork',
        'kernel_sizze': (3, 3),
        'activation_fn': {
            'name': 'tanh'
        }
    },
    'optimizer': {
        'name': 'Adagrad'
    }
}
cfgs = [cfg1, cfg2, cfg_wrong_model_name, cfg_wrong_kernel_size, cfg_wrong_kernel_size_name]

In [86]:
for cfg in cfgs:
    try:
        cfg = ExperimentConfig(**cfg)
        print(cfg)

        model = cfg.model.build()
        print(model) 
        print(model.activation_fn)
        optimizer = cfg.optimizer.build()
        print(optimizer)
        
    except ValidationError as e:
        # thrown by pydantic
        print(e)
    except ValueError as e:
        # thrown by typeconf
        print(e)
    print('\n\n')

model=LinearModelConfig(name='LinearNetwork', activation_fn=Sigmoid(name='sigmoid'), in_features=10, out_features=10) optimizer=AdagradConfig(name='Adagrad', lr=0.01, lr_decay=0, eps=1e-10, initial_accumulator_value=0.0, weight_decay=0)
LinearModel(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
)
<built-in method sigmoid of type object at 0x11137fc30>



model=ConvModelConfig(name='ConvNetwork', activation_fn=Tanh(name='tanh'), kernel_size=(3, 3)) optimizer=AdagradConfig(name='Adagrad', lr=0.01, lr_decay=0, eps=1e-10, initial_accumulator_value=0.0, weight_decay=0)
ConvModel(
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
)
<built-in method tanh of type object at 0x11137fc30>



Unknown option for ModelConfig: ConNetwork



2 validation errors for ExperimentConfig
model -> kernel_size
  wrong tuple length 3, expected 2 (type=value_error.tuple.length; actual_length=3; expected_length=2)
model -> kernel_size
  value is not a valid integer (type=type_error.integer